In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [8]:
import mlflow

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

<Experiment: artifact_location='mlflow-artifacts:/106076302893227074', creation_time=1718905080887, experiment_id='106076302893227074', last_update_time=1718905080887, lifecycle_stage='active', name='MLflow Quickstart', tags={}>

In [9]:
# Load the datasets
fraud_data = pd.read_csv('../data/raw/Fraud_Data.csv')
ip_data = pd.read_csv('../data/raw/IpAddress_to_Country.csv')
creditcard_data = pd.read_csv('../data/raw/creditcard.csv')

# Convert datetime strings to datetime objects
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

# Extract useful datetime components
fraud_data['signup_hour'] = fraud_data['signup_time'].dt.hour
fraud_data['signup_day'] = fraud_data['signup_time'].dt.dayofweek
fraud_data['purchase_hour'] = fraud_data['purchase_time'].dt.hour
fraud_data['purchase_day'] = fraud_data['purchase_time'].dt.dayofweek

# Drop the original datetime columns
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])


In [10]:
# For fraud_data
fraud_X = fraud_data.drop(columns=['class'])
fraud_y = fraud_data['class']
# For creditcard_data
creditcard_X = creditcard_data.drop(columns=['Class'])
creditcard_y = creditcard_data['Class']

# Split fraud_data
fraud_X_train, fraud_X_test, fraud_y_train, fraud_y_test = train_test_split(fraud_X, fraud_y, test_size=0.3, random_state=42)

# Split creditcard_data
creditcard_X_train, creditcard_X_test, creditcard_y_train, creditcard_y_test = train_test_split(creditcard_X, creditcard_y, test_size=0.3, random_state=42)

In [11]:
# Define preprocessing for numerical and categorical features
numeric_features = ['purchase_value', 'age']  # Example numeric features
categorical_features = ['source', 'browser', 'sex', 'signup_hour', 'signup_day', 'purchase_hour', 'purchase_day']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [57]:
# Model Selection
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'MLP': MLPClassifier()
}

# Function to train and evaluate models for fraud data
def train_evaluate_model_fraud(model_name, model, X_train, X_test, y_train, y_test):
    # Create a pipeline with preprocessing and model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Infer model signature
    signature = infer_signature(X_train, y_pred)

    # Log metrics and model with MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.set_tag("Training Info", "All models for Fraud detection")
        model_info = mlflow.sklearn.log_model(
            sk_model=pipeline,
            artifact_path="fraud_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"{model_name}_fraud_detection",
        )

    return {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

# Function to train and evaluate models for credit card data
def train_evaluate_model_credit(model_name, model, X_train, X_test, y_train, y_test):
    # Create a pipeline with preprocessing and model
    pipeline = Pipeline(steps=[('classifier', model)])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Infer model signature
    signature = infer_signature(X_train, y_pred)

    # Log metrics and model with MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.set_tag("Training Info", "All models for Credit Card Fraud detection")
        model_info = mlflow.sklearn.log_model(
            sk_model=pipeline,
            artifact_path="creditcard_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"{model_name}_creditcard_detection",
        )

    return {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

In [13]:
# Train and evaluate models on fraud data
fraud_results = []
for model_name, model in models.items():
    fraud_results.append(train_evaluate_model_fraud(model_name, model, fraud_X_train, fraud_X_test, fraud_y_train, fraud_y_test))

# Train and evaluate models on credit card data
creditcard_results = []
for model_name, model in models.items():
    creditcard_results.append(train_evaluate_model_credit(model_name, model, creditcard_X_train, creditcard_X_test, creditcard_y_train, creditcard_y_test))

# Display results
fraud_results_df = pd.DataFrame(fraud_results)
creditcard_results_df = pd.DataFrame(creditcard_results)

print("Fraud Data Results:\n", fraud_results_df)
print("Credit Card Data Results:\n", creditcard_results_df)

c:\Users\Ted\Desktop\Juju\DEV\fraud_detection_model\venv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ted\Desktop\Juju\DEV\fraud_detection_model\venv\Lib\site-packages\mlflow\types\utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing 

Fraud Data Results:
                  model  accuracy  precision    recall  f1_score
0  Logistic Regression  0.906979   0.000000  0.000000  0.000000
1        Decision Tree  0.908987   0.509699  0.566991  0.536821
2        Random Forest  0.957052   0.993050  0.542092  0.701335
3    Gradient Boosting  0.908457   0.935065  0.017074  0.033535
4                  MLP  0.948185   0.839884  0.547309  0.662742
Credit Card Data Results:
                  model  accuracy  precision    recall  f1_score
0  Logistic Regression  0.998666   0.577465  0.602941  0.589928
1        Decision Tree  0.999169   0.707006  0.816176  0.757679
2        Random Forest  0.999602   0.939655  0.801471  0.865079
3    Gradient Boosting  0.998584   0.894737  0.125000  0.219355
4                  MLP  0.995904   0.257919  0.838235  0.394464


Created version '1' of model 'MLP_creditcard_detection'.


In [22]:
# Load a model for inference
logged_model = 'runs:/79c1280bbcc3459c8f2584cf2d14219d/fraud_model'

# Load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Use the loaded model to make predictions
sample_input = fraud_X_test.iloc[:5]
predictions = loaded_model.predict(sample_input)
print("Predictions:\n", predictions)

Predictions:
 [0 0 0 0 0]


In [65]:
# Making Prediction by using the loaded model for Fraud data
import mlflow
import pandas as pd

models_runs = [
    ('fraud_model', '79c1280bbcc3459c8f2584cf2d14219d'),
    ('fraud_model', '3548a18afe3e4b038e16d5f989adaa38'),
    ('fraud_model', '1d316e8924224d38abaf5b4d39bdfb86'),
    ('fraud_model', '539c77b3762941818b8ec46320d9ae8c'),
    ('fraud_model', '2886dc3a43a845b4acaa4c1b84662cf9')
]
sample_input = fraud_X_test.iloc[:5]

results_dfs = {}

for model_name, run_id in models_runs:
    logged_model = f'runs:/{run_id}/{model_name}'
    
    # Load the model as a PyFuncModel
    loaded_model = mlflow.pyfunc.load_model(logged_model)
    
    # Make predictions
    predictions = loaded_model.predict(sample_input)

    # Create a DataFrame for the predictions
    results_df = pd.DataFrame({
        'Model': model_name,
        'Run ID': run_id,
        'Index': range(len(predictions)),
        'Prediction': predictions
    })
    
    # Store the DataFrame in the dictionary
    results_dfs[f"{model_name}_{run_id}"] = results_df
    
# Display each DataFrame
for key, df in results_dfs.items():
    print(f"Predictions for {key}:\n{df}\n")


Predictions for fraud_model_79c1280bbcc3459c8f2584cf2d14219d:
         Model                            Run ID  Index  Prediction
0  fraud_model  79c1280bbcc3459c8f2584cf2d14219d      0           0
1  fraud_model  79c1280bbcc3459c8f2584cf2d14219d      1           0
2  fraud_model  79c1280bbcc3459c8f2584cf2d14219d      2           0
3  fraud_model  79c1280bbcc3459c8f2584cf2d14219d      3           0
4  fraud_model  79c1280bbcc3459c8f2584cf2d14219d      4           0

Predictions for fraud_model_3548a18afe3e4b038e16d5f989adaa38:
         Model                            Run ID  Index  Prediction
0  fraud_model  3548a18afe3e4b038e16d5f989adaa38      0           0
1  fraud_model  3548a18afe3e4b038e16d5f989adaa38      1           0
2  fraud_model  3548a18afe3e4b038e16d5f989adaa38      2           0
3  fraud_model  3548a18afe3e4b038e16d5f989adaa38      3           0
4  fraud_model  3548a18afe3e4b038e16d5f989adaa38      4           0

Predictions for fraud_model_1d316e8924224d38abaf5b4d39bdfb

In [ ]:
import mlflow
import pandas as pd

models_runs = [
    ('fraud_model', '79c1280bbcc3459c8f2584cf2d14219d'),
    ('fraud_model', '3548a18afe3e4b038e16d5f989adaa38'),
    ('fraud_model', '1d316e8924224d38abaf5b4d39bdfb86'),
    ('fraud_model', '539c77b3762941818b8ec46320d9ae8c'),
    ('fraud_model', '2886dc3a43a845b4acaa4c1b84662cf9'),
    ('creditcard_model', 'e791f0b0d76849ca9c142fb6651140e2'),
    ('creditcard_model', '903c9fddc6ba425ba10e393e5b172282'),
    ('creditcard_model', '5ab4536b36814340bcce64d2db6fb5aa'),
    ('creditcard_model', '5c46165fea0d41999184d09b35498264'),
    ('creditcard_model', 'bf46d67943534f2f8d196db97499b71b')
]

sample_input = fraud_X_test.iloc[:5]

In [66]:
# Making Prediction by using the loaded model for Credit card data
import mlflow
import pandas as pd

models_runs = [
    ('creditcard_model', 'e791f0b0d76849ca9c142fb6651140e2'),
    ('creditcard_model', '903c9fddc6ba425ba10e393e5b172282'),
    ('creditcard_model', '5ab4536b36814340bcce64d2db6fb5aa'),
    ('creditcard_model', '5c46165fea0d41999184d09b35498264'),
    ('creditcard_model', 'bf46d67943534f2f8d196db97499b71b')
]
sample_input = creditcard_X_test.iloc[:5]

results_dfs = {}

for model_name, run_id in models_runs:
    logged_model = f'runs:/{run_id}/{model_name}'
    
    # Load the model as a PyFuncModel
    loaded_model = mlflow.pyfunc.load_model(logged_model)
    
    # Make predictions
    predictions = loaded_model.predict(sample_input)

    # Create a DataFrame for the predictions
    results_df = pd.DataFrame({
        'Model': model_name,
        'Run ID': run_id,
        'Index': range(len(predictions)),
        'Prediction': predictions
    })
    
    # Store the DataFrame in the dictionary
    results_dfs[f"{model_name}_{run_id}"] = results_df
    
# Display each DataFrame
for key, df in results_dfs.items():
    print(f"Predictions for {key}:\n{df}\n")

Predictions for creditcard_model_e791f0b0d76849ca9c142fb6651140e2:
              Model                            Run ID  Index  Prediction
0  creditcard_model  e791f0b0d76849ca9c142fb6651140e2      0           1
1  creditcard_model  e791f0b0d76849ca9c142fb6651140e2      1           0
2  creditcard_model  e791f0b0d76849ca9c142fb6651140e2      2           0
3  creditcard_model  e791f0b0d76849ca9c142fb6651140e2      3           0
4  creditcard_model  e791f0b0d76849ca9c142fb6651140e2      4           0

Predictions for creditcard_model_903c9fddc6ba425ba10e393e5b172282:
              Model                            Run ID  Index  Prediction
0  creditcard_model  903c9fddc6ba425ba10e393e5b172282      0           1
1  creditcard_model  903c9fddc6ba425ba10e393e5b172282      1           0
2  creditcard_model  903c9fddc6ba425ba10e393e5b172282      2           0
3  creditcard_model  903c9fddc6ba425ba10e393e5b172282      3           0
4  creditcard_model  903c9fddc6ba425ba10e393e5b172282      4  